In [1]:
import joblib
import sagemaker
import sagemaker
import joblib
import sklearn
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer, JSONDeserializer
from sagemaker.model_monitor import DataCaptureConfig


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!pip uninstall scikit-learn -y
!pip install scikit-learn=='1.2.1'

Found existing installation: scikit-learn 1.3.2
Uninstalling scikit-learn-1.3.2:
  Successfully uninstalled scikit-learn-1.3.2
  Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-features 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-tabular 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.


In [3]:
session = sagemaker.Session()
bucket = session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [33]:
%%writefile entrypoint.py
import joblib
import os
import json
from sagemaker_containers.beta.framework import (content_types, encoders, env, modules, transformer, worker)

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.jobli"))
    return clf

def predict_fn(input_data, model):
     if len(input_data.shape) == 1:
      proba = model.predict_proba(input_data.reshape(-1, 1).T)
     else:
      proba = model.predict_proba(input_data)   
     return [proba[0][1]]


def output_fn(prediction, accept):

    if accept == "application/json":
        return worker.Response(json.dumps(prediction[0]), mimetype=accept)
    elif accept == 'text/csv':
        #return worker.Response(encoders.encode(prediction[0], accept), mimetype=accept)
        return worker.Response(json.dumps(prediction[0]), mimetype=accept)


Overwriting entrypoint.py


In [6]:
!python -c "from entrypoint import *; import numpy as np; model = model_fn(''); print(predict_fn(np.array([[17,11,0,240,241,0,1,123,1,0.2,1,0,3]]), model))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/sagemaker-user/monitoreo/entrypoint.py", line 4, in <module>
    from sagemaker_containers.beta.framework import (content_types, encoders, env, modules, transformer, worker)
ModuleNotFoundError: No module named 'sagemaker_containers'


In [34]:
model_uri = 's3://sagemaker-us-east-1-381492271173/custom_model/heart/model.tar.gz'
sklearn_model = SKLearnModel(model_data=model_uri,
                             role="LabRole",
                             entry_point="entrypoint.py",
                             framework_version="1.2-1")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
%store -r path

In [9]:
data_capture_uri = f"s3://{bucket}/{path}/data-capture"

In [10]:
data_capture_uri

's3://sagemaker-us-east-1-381492271173/monitor/heart/data-capture'

In [35]:
data_capture_config = DataCaptureConfig(enable_capture=True,
                                        sampling_percentage=100,
                                        destination_s3_uri=data_capture_uri,
                                        csv_content_types=["text/csv"]
                                    )
                                        

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [36]:
predictor = sklearn_model.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.xlarge',
                                 serielizer=CSVSerializer(),
                                 deserializer=CSVDeserializer(),
                                 data_capture_config=data_capture_config)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
-----!

In [32]:
#predictor.delete_endpoint()

In [40]:
endpoint_name = predictor.endpoint_name

In [38]:
%store endpoint_name

Stored 'endpoint_name' (str)


In [41]:
endpoint_name

'sagemaker-scikit-learn-2024-03-21-00-03-05-484'